In [1]:
import pandas as pd
import datetime
import seaborn as sb
import time
from ipywidgets import interact, fixed
from libs.CovidTimeseriesModel import CovidTimeseriesModel
from libs.CovidDatasets import CovidDatasets
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [2]:
Dataset = CovidDatasets()

def model_state(country, state, interventions=None):
   
    ## Constants
    start_time = time.time()
    HOSPITALIZATION_RATE = .0727
    HOSPITALIZED_CASES_REQUIRING_ICU_CARE = .1397
    TOTAL_INFECTED_PERIOD = 12
    MODEL_INTERVAL = 4
    r0 = 2.4
    POP = Dataset.get_population_by_country_state(country, state)
    # Pack all of the assumptions and parameters into a dict that can be passed into the model
    MODEL_PARAMETERS = {
        # Pack the changeable model parameters
        'timeseries': Dataset.get_timeseries_by_country_state(country, state, MODEL_INTERVAL),
        'beds': Dataset.get_beds_by_country_state(country, state),
        'population': POP,
        'projection_iterations': 25, # Number of iterations into the future to project
        'r0': r0,
        'interventions': interventions,
        'hospitalization_rate': HOSPITALIZATION_RATE,
        'case_fatality_rate': .0109341104294479,
        'hospitalized_cases_requiring_icu_care': HOSPITALIZED_CASES_REQUIRING_ICU_CARE,
        # Assumes that anyone who needs ICU care and doesn't get it dies
        'case_fatality_rate_hospitals_overwhelmed': HOSPITALIZATION_RATE * HOSPITALIZED_CASES_REQUIRING_ICU_CARE,
        'hospital_capacity_change_daily_rate': 1.05,
        'max_hospital_capacity_factor': 2.07,
        'initial_hospital_bed_utilization': .6,
        'model_interval': 4, # In days
        'total_infected_period': 12, # In days
        'rolling_intervals_for_current_infected': int(round(TOTAL_INFECTED_PERIOD / MODEL_INTERVAL, 0)),
        'estimated_new_cases_per_death': 32,
        'estimated_new_cases_per_confirmed': 20
    }
    return CovidTimeseriesModel().forecast_region(model_parameters=MODEL_PARAMETERS)

r0 = 2.4

INTERVENTIONS = [
    None, 
    {
        datetime.date(2020, 3, 23): 1.3,
        datetime.date(2020, 4, 20): 1.1,
        datetime.date(2020, 5, 22): 0.8,
        datetime.date(2020, 6, 23): r0
    },
    {
        datetime.date(2020, 3, 23): 1.7,
        datetime.date(2020, 6, 23): r0
    },
    {
        datetime.date(2020, 3, 23): 1.3,
        datetime.date(2020, 3, 31): 0.3,
        datetime.date(2020, 4, 28): 0.2,
        datetime.date(2020, 5,  6): 0.1,
        datetime.date(2020, 5, 10): 0.35,
        datetime.date(2020, 5, 18): r0
    }
]

In [3]:
Dataset.get_timeseries_by_country_state('USA', 'CA', 1)[['date','cases','deaths']]

,date,cases,deaths
0,2020-03-03,25.00,0.00
1,2020-03-04,35.00,1.00
2,2020-03-05,51.00,1.00
3,2020-03-06,59.00,1.00
4,2020-03-07,81.00,1.00
5,2020-03-08,95.00,1.00
6,2020-03-09,101.00,1.00
7,2020-03-10,144.00,2.00
8,2020-03-11,177.00,3.00
9,2020-03-12,221.00,4.00


In [5]:
model_state('USA', 'TX', INTERVENTIONS[0])

{'active': 56.0,
 'cases': 56.0,
 'city': nan,
 'country': 'USA',
 'county': nan,
 'date': Timestamp('2020-03-15 00:00:00'),
 'deaths': nan,
 'effective_r': nan,
 'growthFactor': 1.0980392156862746,
 'i': 3,
 'index': 2945,
 'lat': 31.177528238191037,
 'level_0': 10,
 'long': -100.0884705015624,
 'population': 28995881.0,
 'r': 2.6666666666666665,
 'recovered': nan,
 'state': 'TX',
 'synthetic': None,
 'tested': nan,
 'url': 'https://covidtracking.com/api/states'}


KeyError: 'newly_infected'